In [1]:
import asyncio
import time
import datetime
from indy import anoncreds, did, ledger, pool, wallet, IndyError

import json

async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'], identity['wallet_credentials'])


async def getting_verinym(from_, to):
    await create_wallet(to)

    (to['did'], to['key']) = await did.create_and_store_my_did(to['wallet'], "{}")

    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }

    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])


async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)


async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)


async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)


async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    get_schema_response = await ledger.submit_request(pool_handle, get_schema_request)
    return await ledger.parse_get_schema_response(get_schema_response)


async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = await ledger.submit_request(pool_handle, get_cred_def_request)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)


async def get_credential_for_referent(search_handle, referent):
    credentials = json.loads(
        await anoncreds.prover_fetch_credentials_for_proof_req(search_handle, referent, 10))
    return credentials[0]['cred_info']


async def prover_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_states = {}
    for item in identifiers.values():
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Create Revocation States

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_states)


async def verifier_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_reg_defs = {}
    rev_regs = {}
    for item in identifiers:
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Get Revocation Definitions and Revocation Registries

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_reg_defs), json.dumps(rev_regs)

In [2]:
async def run():
    N = 5
    print("Certificate creation benchmark -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    print("==============================")
    print("=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==")
    print("------------------------------")

    steward = {
        'name': "Sovrin Steward",
        'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
        'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
        'pool': pool_['handle'],
        'seed': '000000000000000000000000Steward1'
    }

    await create_wallet(steward)

    print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
    steward['did_info'] = json.dumps({'seed': steward['seed']})
    steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

    government = {
        'name': 'Government',
        'wallet_config': json.dumps({'id': 'government_wallet'}),
        'wallet_credentials': json.dumps({'key': 'government_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }

    await getting_verinym(steward, government)

    faber = {
        'name': 'Faber',
        'wallet_config': json.dumps({'id': 'faber_wallet'}),
        'wallet_credentials': json.dumps({'key': 'faber_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }

    await getting_verinym(steward, faber)


    print("==============================")
    print("=== Credential Schemas Setup ==")
    print("------------------------------")

    print("\"Government\" -> Create \"Transcript\" Schema")
    transcript = {
        'name': 'Transcript',
        'version': '1.2',
        'attributes': ['first_name', 'last_name', 'degree', 'status', 'year', 'average', 'ssn']
    }
    (government['transcript_schema_id'], government['transcript_schema']) = \
        await anoncreds.issuer_create_schema(government['did'], transcript['name'], transcript['version'],
                                             json.dumps(transcript['attributes']))
    transcript_schema_id = government['transcript_schema_id']

    print("\"Government\" -> Send \"Transcript\" Schema to Ledger")
    await send_schema(government['pool'], government['wallet'], government['did'], government['transcript_schema'])

    time.sleep(1)  # sleep 1 second before getting schema

    print("==============================")
    print("=== Faber Credential Definition Setup ==")
    print("------------------------------")

    print("\"Faber\" -> Get \"Transcript\" Schema from Ledger")
    (faber['transcript_schema_id'], faber['transcript_schema']) = \
        await get_schema(faber['pool'], faber['did'], transcript_schema_id)

    print("\"Faber\" -> Create and store in Wallet \"Faber Transcript\" Credential Definition")
    transcript_cred_def = {
        'tag': 'TAG1',
        'type': 'CL',
        'config': {"support_revocation": False}
    }
    (faber['transcript_cred_def_id'], faber['transcript_cred_def']) = \
        await anoncreds.issuer_create_and_store_credential_def(faber['wallet'], faber['did'],
                                                               faber['transcript_schema'], transcript_cred_def['tag'],
                                                               transcript_cred_def['type'],
                                                               json.dumps(transcript_cred_def['config']))

    print("\"Faber\" -> Send  \"Faber Transcript\" Credential Definition to Ledger")
    await send_cred_def(faber['pool'], faber['wallet'], faber['did'], faber['transcript_cred_def'])

   

    print("==============================")
    print("=== Getting Transcript with Faber ==")
    print("==============================")
    print("== Alice setup ==")
    print("==============================")

    alice = {
        'name': 'Alice',
        'wallet_config': json.dumps({'id': 'alice_wallet'}),
        'wallet_credentials': json.dumps({'key': 'alice_wallet_key'}),
        'pool': pool_['handle'],
    }
    await create_wallet(alice)
    (alice['did'], alice['key']) = await did.create_and_store_my_did(alice['wallet'], "{}")

    print("==============================")
    print("== Getting Transcript with Faber - Getting Transcript Credential ==")
    print("------------------------------")

    creation_time = []

    for i in range(N):
        print("Certificate creation - " + str(i))
        start = datetime.datetime.now()

        print("\"Faber\" -> Create \"Transcript\" Credential Offer for Alice")
        faber['transcript_cred_offer'] = \
            await anoncreds.issuer_create_credential_offer(faber['wallet'], faber['transcript_cred_def_id'])

        print("\"Faber\" -> Send \"Transcript\" Credential Offer to Alice")
        alice['transcript_cred_offer'] = faber['transcript_cred_offer']
        transcript_cred_offer_object = json.loads(alice['transcript_cred_offer'])

        alice['transcript_schema_id'] = transcript_cred_offer_object['schema_id']
        alice['transcript_cred_def_id'] = transcript_cred_offer_object['cred_def_id']

        print("\"Alice\" -> Create and store \"Alice\" Master Secret in Wallet")
        alice['master_secret_id'] = await anoncreds.prover_create_master_secret(alice['wallet'], None)

        print("\"Alice\" -> Get \"Faber Transcript\" Credential Definition from Ledger")
        (alice['faber_transcript_cred_def_id'], alice['faber_transcript_cred_def']) = \
            await get_cred_def(alice['pool'], alice['did'], alice['transcript_cred_def_id'])

        print("\"Alice\" -> Create \"Transcript\" Credential Request for Faber")
        (alice['transcript_cred_request'], alice['transcript_cred_request_metadata']) = \
            await anoncreds.prover_create_credential_req(alice['wallet'], alice['did'],
                                                        alice['transcript_cred_offer'], alice['faber_transcript_cred_def'],
                                                        alice['master_secret_id'])

        print("\"Alice\" -> Send \"Transcript\" Credential Request to Faber")
        faber['transcript_cred_request'] = alice['transcript_cred_request']


        alice['transcript_cred_values'] = json.dumps({
            "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
            "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
            "degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
            "status": {"raw": "graduated", "encoded": "2213454313412354"},
            "ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
            "year": {"raw": "2015", "encoded": "2015"},
            "average": {"raw": "5", "encoded": "5"}
        })
        faber['alice_transcript_cred_values'] = alice['transcript_cred_values']

        print("\"Faber\" -> Create \"Transcript\" Credential for Alice")

        faber['transcript_cred'], _, _ = \
            await anoncreds.issuer_create_credential(faber['wallet'], faber['transcript_cred_offer'],
                                                    faber['transcript_cred_request'],
                                                    faber['alice_transcript_cred_values'], None, None)

        print("\"Faber\" -> Send \"Transcript\" Credential to Alice")
        alice['transcript_cred'] = faber['transcript_cred']

        print("\"Alice\" -> Store \"Transcript\" Credential from Faber")
        _, alice['transcript_cred_def'] = await get_cred_def(alice['pool'], alice['did'],
                                                            alice['transcript_cred_def_id'])

        await anoncreds.prover_store_credential(alice['wallet'], None, alice['transcript_cred_request_metadata'],
                                                alice['transcript_cred'], alice['transcript_cred_def'], None)
        
        end = datetime.datetime.now()

        differance = end - start
        creation_time.append(differance.microseconds)


    print("==============================")

    print(" \"Sovrin Steward\" -> Close and Delete wallet")
    await wallet.close_wallet(steward['wallet'])
    await wallet.delete_wallet(steward['wallet_config'], steward['wallet_credentials'])

    print("\"Government\" -> Close and Delete wallet")
    await wallet.close_wallet(government['wallet'])
    await wallet.delete_wallet(government['wallet_config'], government['wallet_credentials'])

    print("\"Faber\" -> Close and Delete wallet")
    await wallet.close_wallet(faber['wallet'])
    await wallet.delete_wallet(faber['wallet_config'], faber['wallet_credentials'])

    print("\"Alice\" -> Close and Delete wallet")
    await wallet.close_wallet(alice['wallet'])
    await wallet.delete_wallet(alice['wallet_config'], alice['wallet_credentials'])

    print("Close and Delete pool")
    await pool.close_pool_ledger(pool_['handle'])
    await pool.delete_pool_ledger_config(pool_['name'])

    print("Certificate creation benchmark -> done")
    
    print(" -- Creation time -- ")
    creation_time_string = "["
    for i in creation_time:
        creation_time_string += str(i)+","
    creation_time_string = creation_time_string[:-1]
    creation_time_string +="]"
    print(creation_time_string)


if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(run())
    time.sleep(1)  # FIXME waiting for libindy thread complete


Certificate creation benchmark -> started
Open Pool Ledger: pool1
=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==
------------------------------
"Sovrin Steward" -> Create wallet
"Sovrin Steward" -> Create and store in Wallet DID from seed
"Government" -> Create wallet
"Faber" -> Create wallet
=== Credential Schemas Setup ==
------------------------------
"Government" -> Create "Transcript" Schema
"Government" -> Send "Transcript" Schema to Ledger
=== Faber Credential Definition Setup ==
------------------------------
"Faber" -> Get "Transcript" Schema from Ledger
"Faber" -> Create and store in Wallet "Faber Transcript" Credential Definition
"Faber" -> Send  "Faber Transcript" Credential Definition to Ledger
=== Getting Transcript with Faber ==
== Alice setup ==
"Alice" -> Create wallet
== Getting Transcript with Faber - Getting Transcript Credential ==
------------------------------
"Faber" -> Create "Transcript" Credential Offer for Alice
"Faber" -> Se